In [1]:
from datetime import date

In [2]:
now = date.today()
now

datetime.date(2022, 10, 25)

In [3]:
customer_id = '0140'

In [4]:
query = f'''http://192.168.1.104/ReportServer/Pages/ReportViewer.aspx\
?%2fDynamicsAX%2fCustPriceList.PrecisionDesign&CustPriceList_CustAccount={customer_id}\
&CustPriceList_PerDate={now}&CustPriceList_CurrencyCode=usd\
&CustPriceList_DataArea=dka&rs:Format=Excel'''

In [5]:
query

'http://192.168.1.104/ReportServer/Pages/ReportViewer.aspx?%2fDynamicsAX%2fCustPriceList.PrecisionDesign&CustPriceList_CustAccount=0140&CustPriceList_PerDate=2022-10-24&CustPriceList_CurrencyCode=usd&CustPriceList_DataArea=dka&rs:Format=Excel'

In [32]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [40]:
query = os.getenv('QUERY')

In [41]:
query

'http://192.168.1.104/ReportServer/Pages/ReportViewer.aspx?%2fDynamicsAX%2fCustPriceList.PrecisionDesign&CustPriceList_CustAccount={customer_id}&CustPriceList_PerDate={now}&CustPriceList_CurrencyCode=usd&CustPriceList_DataArea=dka&rs:Command=Render&rs:Format=Excel'

In [7]:
import requests
from requests_ntlm import HttpNtlmAuth

In [8]:
username = os.getenv('USERNAME_REPORT_SERVER')
password = os.getenv('PASSWORD_REPORT_SERVER')

In [9]:
username, password

('report_server', 'YandexNavsegda1212')

In [10]:
from io import BytesIO

In [11]:
request = requests.get(query,auth=HttpNtlmAuth(username, password))

In [12]:
request.status_code

200

In [13]:
fp = BytesIO(request.content)

In [ ]:
if request.status_code == 200:
    with open('price_list.xlsx', 'wb') as out:
        for bits in request.iter_content():
            out.write(bits)

In [4]:
import pandas as pd
#from StyleFrame import StyleFrame, Styler, utils

In [24]:
df = pd.read_excel(request.content, skiprows=10)
df = df.assign(Order='',Order_Sum='')

WARNING *** file size (81792) not 512 + multiple of sector size (512)


In [25]:
df.head()

,Brand,Category 1,Category 2,Unnamed: 3,Category 3,Category 4,Unnamed: 6,Unnamed: 7,Model,Name,...,Price,Unnamed: 19,Reserved,Discounts / Скидки,Unnamed: 22,Gifts / Подарки,Unnamed: 24,Reserved total,Order,Order_Sum
0,BEKO,Home Appliances,Vacuum Cleaner,NaN,Vacuum Cleaner Bag,NaN,NaN,NaN,VCC6424 WI,Vacuum Cleaner Beko VCC6424 WI,...,57.5,NaN,NaN,NaN,NaN,NaN,NaN,0,,
1,BEKO,Home Appliances,Vacuum Cleaner,NaN,Vacuum Cleaner Bag,NaN,NaN,NaN,VCC7324 WI,Vacuum Cleaner Beko VCC7324 WI,...,66.0,NaN,NaN,NaN,NaN,NaN,NaN,0,,
2,Chuwi,Computers,NaN,NaN,NaN,NaN,NaN,NaN,CoreBox,mini-PC CoreBox i5-8259U/16GB/512GB,...,355.3,NaN,NaN,NaN,NaN,NaN,NaN,0,,
3,Chuwi,Computers,NaN,NaN,NaN,NaN,NaN,NaN,HeroBox,mini-PC HeroBox J4125/8GB/256GB,...,180.4,NaN,NaN,NaN,NaN,NaN,NaN,0,,
4,Chuwi,Computers,Laptops,NaN,Notebooks,NaN,NaN,NaN,GemiBook Pro,"Laptop GemiBook Pro 14""/N5100/8GB/256GB",...,290.0,NaN,NaN,NaN,NaN,NaN,NaN,0,,


In [5]:
df = pd.read_excel('price_list.xlsx', skiprows=10, usecols="A:C,E,F,I,J,M:S,U", engine='xlrd')
df = df.assign(Order='',Order_Sum='')

WARNING *** file size (89984) not 512 + multiple of sector size (512)


In [ ]:
# [opt for opt in dir(request) if not opt.startswith("__")]

In [6]:
writer = pd.ExcelWriter(f'Price_list-{now}.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Price_list', index=False, startrow=5)

In [7]:
workbook  = writer.book
worksheet = writer.sheets['Price_list']

In [8]:
header_format = workbook.add_format({
    'bold': True,
    # 'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'fg_color': '#41E3A1',
    'border': 1})

In [9]:
total_format = workbook.add_format({
    'bold': True,
    'border': 1})

In [10]:
small_header_format = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'fg_color': '#41E3A1'})

In [11]:
money_format = workbook.add_format({'num_format': '$# ##0.##',
'border': 1
})

In [12]:
money_format_ = workbook.add_format({'num_format': '$# ##0.##'
})

In [13]:
cell_format = workbook.add_format({'border': 1,
'num_format': '# ##0.##'})

In [14]:
for col_num, value in enumerate(df.columns.values):
    worksheet.write(5, col_num, value, header_format)

In [15]:
x = (10,13,13,13,16,16,30,13,13,14,12,13,12,14,14,12,19)
for i,x in enumerate(x):
    worksheet.set_column(i,i,x)

In [16]:
worksheet.set_column(13, 13, 14, money_format_)

0

In [17]:
worksheet.write_string('A4', 'Customer:')
worksheet.write_string('B4', f'{customer_id}')
worksheet.write_string('A5', 'Currency:')
worksheet.write_string('B5', 'USD')

0

In [18]:
# Tables with reserve totals
worksheet.write_string('N2', 'Total weight', total_format)
worksheet.write_string('N3', 'Total CBM', total_format)
worksheet.write_string('N4', 'Total Cartoons', total_format)
worksheet.write_string('N5', 'Total Sum', total_format)

0

In [19]:
# Small table's header
worksheet.write_string('N1', '', small_header_format)
worksheet.write_string('O1', 'Reserved', small_header_format)
worksheet.write_string('P1', 'Order', small_header_format)
worksheet.write_string('Q1', 'Total Res + Ord', small_header_format)

0

In [20]:
# add formulas for reserved goods
row_numbers = df.shape[0] + 1
worksheet.write_dynamic_array_formula('O2', f'=SUM($I$7:$I${row_numbers}*$O$7:$O${row_numbers})',cell_format) # Total weight
worksheet.write_dynamic_array_formula('O3', f'=SUM($J$7:$J${row_numbers}*$O$7:$O${row_numbers})',cell_format) # Total CBM
worksheet.write_dynamic_array_formula('O4', f'=SUM($K$7:$K${row_numbers}*$O$7:$O${row_numbers})') # Total Cartoons
worksheet.write_dynamic_array_formula('O5', f'=SUM($N$7:$N${row_numbers}*$O$7:$O${row_numbers})',money_format) # Total SUM

0

In [21]:
# add formulas for ordered goods
worksheet.write_dynamic_array_formula('P2', f'=SUM($I$7:$I${row_numbers}*$P$7:$P${row_numbers})',cell_format) # Total weight
worksheet.write_dynamic_array_formula('P3', f'=SUM($J$7:$J${row_numbers}*$P$7:$P${row_numbers})',cell_format) # Total CBM
worksheet.write_dynamic_array_formula('P4', f'=SUM($K$7:$K${row_numbers}*$P$7:$P${row_numbers})') # Total Cartoons
worksheet.write_dynamic_array_formula('P5', f'=SUM($N$7:$N${row_numbers}*$P$7:$P${row_numbers})',money_format) # Total SUM

0

In [22]:
worksheet.write_dynamic_array_formula(f'Q7:Q{row_numbers}',    f'=(N7:N{row_numbers}*P7:P{row_numbers})',money_format_)

0

In [23]:
# add formulas for ordered + reserved goods
worksheet.write_formula('Q2', 'O2+P2',cell_format) # Total weight
worksheet.write_formula('Q3', 'O3+P3',cell_format) # Total CBM
worksheet.write_formula('Q4', 'O4+P4') # Total Cartoons
worksheet.write_formula('Q5', 'O5+P5',money_format) # Total SUM

0

In [24]:
worksheet.freeze_panes(6, 0)

In [25]:
worksheet.autofilter('A6:Q6')

In [26]:
merge_format = workbook.add_format({
    # 'bold':     True,
    'align':    'center',
    'valign':   'vcenter',
    'font_name': 'Calibri',
    'font_size': 25 
})

In [27]:
worksheet.merge_range('A1:L3', f'PRICE LIST {now}',merge_format)

0

In [28]:
worksheet.set_zoom(67)

In [29]:
writer.save()

In [ ]:
file_